In [42]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [43]:
data = pd.read_csv('jobs.csv')

data.head()

,JobID,Jenjang Pendidikan,Kualifikasi Pekerjaan,Posisi Pekerjaan,Gender,Lokasi Pekerjaan(Prov),Lokasi Pekerjaan(Kota),Pengalaman Kerja (bulan),Skill yang dibutuhkan,Gaji,Jenis Pekerjaan,Perusahaan,Tanggal Posting,Kategori Perusahaan,Link,Deskripsi
0,1,Kindergarten,Bachelor Degree,Kindergarten Teacher,General,Jawa Barat,Bogor,24,Bahasa Inggris,0,Full time,PT Tunas Tuju Asa,2023-05-16,Sekolah,PRESCHOOL & KINDERGARTEN TEACHERS FOR KINDERFI...,Available for working in the new academic year...
1,2,Kindergarten,Bachelor Degree,Islamic Education Teachers,General,Banten,Tangerang,24,Bahasa Inggris,0,Full time,PT Kinder Haven Pusaka,2023-05-18,Sekolah,Guru Agama (Religion Student Counselor) - PT K...,"Create environments, activities, and programmi..."
2,3,Kindergarten,Bachelor Degree,Kindergarten Teacher,General,DI Yogyakarta,Sleman,24,NaN,2200000,Full time,Yayasan Pendidikan Blue Dolphin International,2023-05-18,Sekolah,Preschool and Kindergarten Teacher - Yay. Pend...,We are a well-grown English-speaking Pre-schoo...
3,4,Kindergarten,Bachelor Degree,Kindergarten Teacher,General,Jawa Timur,Surabaya,0,Bahasa Inggris,0,Full time,Yayasan Intan Eduka,2023-05-18,Sekolah,KINDERGARTEN TEACHER - Yayasan Intan Eduka - 4...,Preparing and delivering lessons to a range of...
4,5,Elementary School,Diploma Degree,Natural Sciences Teacher,General,Jawa Barat,Depok,0,Bahasa Inggris,2000000,Full time,Pratiwi School Depok,2023-05-17,Sekolah,Science and Mathematics Teachers - Pratiwi Sch...,"Age max 35 years old, Bachelor degree in Educa..."


In [44]:
data.columns

Index(['JobID', 'Jenjang Pendidikan', 'Kualifikasi Pekerjaan',
       'Posisi Pekerjaan', 'Gender', 'Lokasi Pekerjaan(Prov)',
       'Lokasi Pekerjaan(Kota)', 'Pengalaman Kerja (bulan)',
       'Skill yang dibutuhkan', 'Gaji', 'Jenis Pekerjaan', 'Perusahaan',
       'Tanggal Posting', 'Kategori Perusahaan', 'Link', 'Deskripsi'],
      dtype='object')

In [45]:
#subsetting only needed columns and not considering the columns that are not necessary
cols = list(['JobID']+['Posisi Pekerjaan']+ ['Skill yang dibutuhkan'] + ['Jenis Pekerjaan'])
final_lowongan =data[cols]
final_lowongan.columns = ['Job.ID','Position','Skill','Jenis']
final_lowongan.head() 

# Menggunakan heatmap

,Job.ID,Position,Skill,Jenis
0,1,Kindergarten Teacher,Bahasa Inggris,Full time
1,2,Islamic Education Teachers,Bahasa Inggris,Full time
2,3,Kindergarten Teacher,NaN,Full time
3,4,Kindergarten Teacher,Bahasa Inggris,Full time
4,5,Natural Sciences Teacher,Bahasa Inggris,Full time


In [46]:
final_lowongan['Skill'].fillna('Good Communication', inplace=True)

<ipython-input-46-938613c0ee32>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_lowongan['Skill'].fillna('Good Communication', inplace=True)


In [47]:
final_lowongan["Skill"].replace('Bahasa Inggris', 'Proficient in English', inplace=True)

<ipython-input-47-97a07b618ffd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_lowongan["Skill"].replace('Bahasa Inggris', 'Proficient in English', inplace=True)


In [48]:
final_lowongan.head()

,Job.ID,Position,Skill,Jenis
0,1,Kindergarten Teacher,Proficient in English,Full time
1,2,Islamic Education Teachers,Proficient in English,Full time
2,3,Kindergarten Teacher,Good Communication,Full time
3,4,Kindergarten Teacher,Proficient in English,Full time
4,5,Natural Sciences Teacher,Proficient in English,Full time


In [64]:
final_lowongan["pos_skill"] = final_lowongan["Position"].map(str) +" "+ final_lowongan["Skill"] + " "+ final_lowongan["Skill"]
final_lowongan.pos_skill.values
# final_lowongan.pos_skill.values

<ipython-input-64-2b9c7613c6d4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_lowongan["pos_skill"] = final_lowongan["Position"].map(str) +" "+ final_lowongan["Skill"] + " "+ final_lowongan["Skill"]


array(['Kindergarten Teacher Proficient in English Proficient in English',
       'Islamic Education Teachers Proficient in English Proficient in English',
       'Kindergarten Teacher Good Communication Good Communication',
       'Kindergarten Teacher Proficient in English Proficient in English',
       'Natural Sciences Teacher Proficient in English Proficient in English',
       'Math teacher Proficient in English Proficient in English',
       'Physical Education and Health Teachers Good Communication Good Communication',
       'Cultural arts teacher Membaca Alquran, mengoperasikan komputer Membaca Alquran, mengoperasikan komputer',
       'Social Sciences Teacher Membaca Alquran, mengoperasikan komputer Membaca Alquran, mengoperasikan komputer',
       'Civics Teachers Membaca Alquran, mengoperasikan komputer Membaca Alquran, mengoperasikan komputer',
       'It teacher experience in teaching kids experience in teaching kids',
       'It teacher experience in teaching kids exper

In [50]:
# Mengubah semua teks ke huruf kecil

In [51]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [52]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

# Create an object of class PorterStemmer
porter = PorterStemmer()

# Define the stop words
stop_words = set(stopwords.words('english')) 

# Function to preprocess the data
def preprocess_data(data):
    # Lowercasing the data
    data = data.lower()
    # Removing the punctuations
    data = re.sub(r'[^\w\s]', '', data)
    # Stemming the data
    data = porter.stem(data)
    # Removing the stop words
    data = " ".join(word for word in data.split() if word not in stop_words)
    return data

# Preprocessing the job descriptions
final_lowongan["pos_skill"] = final_lowongan["pos_skill"].apply(preprocess_data)

<ipython-input-52-2aa01c357e7a>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_lowongan["pos_skill"] = final_lowongan["pos_skill"].apply(preprocess_data)


In [53]:
data2 = list(final_lowongan["pos_skill"].values)
data2

['kindergarten teacher proficient english proficient english',
 'islamic education teachers proficient english proficient english',
 'kindergarten teacher good communication good commun',
 'kindergarten teacher proficient english proficient english',
 'natural sciences teacher proficient english proficient english',
 'math teacher proficient english proficient english',
 'physical education health teachers good communication good commun',
 'cultural arts teacher membaca alquran mengoperasikan komputer membaca alquran mengoperasikan komput',
 'social sciences teacher membaca alquran mengoperasikan komputer membaca alquran mengoperasikan komput',
 'civics teachers membaca alquran mengoperasikan komputer membaca alquran mengoperasikan komput',
 'teacher experience teaching kids experience teaching kid',
 'teacher experience teaching kids experience teaching kid',
 'teacher experience teaching kids experience teaching kid',
 'teacher experience teaching kids experience teaching kid',
 'tea

In [54]:
# Membuat list_of_descriptions
list_of_descriptions = [desc.split() for desc in data2]

In [55]:
# Membuat dan menyimpan model Word2Vec
model = Word2Vec(list_of_descriptions, vector_size=100, window=5, min_count=1, workers=4)

# Fungsi untuk mendapatkan vektor
def get_vector(word_list):
    vector_sum = sum(model.wv[word] for word in word_list if word in model.wv)
    return vector_sum / len(word_list)

# Membuat vektor pekerjaan
job_vectors = np.array([get_vector(desc) for desc in list_of_descriptions])
job_vectors = np.reshape(job_vectors, (len(job_vectors), -1))  # reshape job vectors to 2D

# Membuat dan melatih model rekomendasi
rec_model = Sequential()
rec_model.add(Dense(128, activation='relu', input_dim=100))
rec_model.add(Dense(64, activation='relu'))
rec_model.add(Dense(len(job_vectors[0]), activation='linear'))  # Output layer

rec_model.compile(loss='mse', optimizer='adam', metrics=['mae'])
rec_model.fit(job_vectors, job_vectors, epochs=50, verbose=1)


Epoch 1/50
3/3 [==============================] - 25s 8ms/step - loss: 7.7600e-06 - mae: 0.0022
Epoch 2/50
3/3 [==============================] - 0s 8ms/step - loss: 5.2632e-06 - mae: 0.0018
Epoch 3/50
3/3 [==============================] - 0s 8ms/step - loss: 5.2459e-06 - mae: 0.0018
Epoch 4/50
3/3 [==============================] - 0s 7ms/step - loss: 4.9879e-06 - mae: 0.0018
Epoch 5/50
3/3 [==============================] - 0s 8ms/step - loss: 4.7371e-06 - mae: 0.0017
Epoch 6/50
3/3 [==============================] - 0s 8ms/step - loss: 4.6508e-06 - mae: 0.0017
Epoch 7/50
3/3 [==============================] - 0s 7ms/step - loss: 4.5918e-06 - mae: 0.0017
Epoch 8/50
3/3 [==============================] - 0s 8ms/step - loss: 4.4909e-06 - mae: 0.0017
Epoch 9/50
3/3 [==============================] - 0s 8ms/step - loss: 4.4631e-06 - mae: 0.0017
Epoch 10/50
3/3 [==============================] - 0s 8ms/step - loss: 4.4332e-06 - mae: 0.0017
Epoch 11/50
3/3 [==============================]

In [69]:
# Fungsi untuk mendapatkan pekerjaan yang mirip
def get_similar_jobs(vector):
    similarities = cosine_similarity([vector], job_vectors)[0]
    indices = np.argsort(similarities)[-3:]  # Mengambil 3 pekerjaan dengan kesamaan tertinggi
    return [data2[i] for i in indices]

# Fungsi untuk memberikan rekomendasi
def recommend_jobs(user_description):
    user_vector = get_vector(user_description.split())
    user_vector = np.reshape(user_vector, (1, -1))  # reshape user vector to 2D
    predicted_vector = rec_model.predict(user_vector)[0]
    recommended_jobs = get_similar_jobs(predicted_vector)
    return recommended_jobs

# Mencoba sistem rekomendasi
print(recommend_jobs("I have experience in math"))

1/1 [==============================] - 0s 26ms/step
['english teacher good communication good commun', 'english teacher good communication good commun', 'english teacher good communication good commun']


In [57]:
loss, mae = rec_model.evaluate(job_vectors, job_vectors)
print('Loss:', loss)
print('MAE:', mae)

3/3 [==============================] - 0s 6ms/step - loss: 2.8759e-06 - mae: 0.0013
Loss: 2.8759322958649136e-06
MAE: 0.0013269843766465783


In [70]:
# Fungsi untuk mendapatkan pekerjaan yang mirip
def get_similar_jobs(vector):
    similarities = cosine_similarity([vector], job_vectors)[0]
    df = pd.DataFrame(data={
        'Job': data['Posisi Pekerjaan'],
        'Location': data['Lokasi Pekerjaan(Kota)'],
        'Similarity': similarities
    })
    df = df.sort_values(by='Similarity', ascending=False)  # Urutkan berdasarkan Similarity
    return df

# Fungsi untuk memberikan rekomendasi
def recommend_jobs(user_description):
    user_vector = get_vector(user_description.split())
    user_vector = np.reshape(user_vector, (1, -1))  # reshape user vector to 2D
    predicted_vector = rec_model.predict(user_vector)[0]
    recommended_jobs = get_similar_jobs(predicted_vector)
    return recommended_jobs

# Mencoba sistem rekomendasi
recommend_jobs("I have experience in math").head()

1/1 [==============================] - 0s 26ms/step


,Job,Location,Similarity
38,English teacher,Jakarta Barat,0.825455
37,English teacher,Ambon,0.825455
36,English teacher,Cileungsi,0.825455
35,English teacher,Tangerang,0.825455
44,it teacher,Samarinda,0.757035
